In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [0]:
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
import numpy as np
from skimage.io import imread
import os
from torchvision import transforms, models
'''
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
'''

In [0]:
train_dir='/content/gdrive/My Drive/датасет_тренировка'
val_dir='/content/gdrive/My Drive/датасет_валидация'
train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

In [0]:

batch_size = 50
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

In [0]:
print(len(train_dataloader), len(train_dataset))

In [0]:
X_batch, y_batch = next(iter(train_dataloader))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
plt.imshow(X_batch[0].permute(1, 2, 0).numpy() * std + mean)
print(y_batch)

In [0]:
class_names=['Cat','Dog']
def show_input(input_tensor, title=''):
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1))
    plt.title(title)
    plt.show()
    plt.pause(0.001)

X_batch, y_batch = next(iter(train_dataloader))

for x_item, y_item in zip(X_batch, y_batch):
    show_input(x_item, title=class_names[y_item])

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Объявление класса модели 

In [0]:
class CatDogNet_2020(torch.nn.Module):
 def __init__(self):
    super(CatDogNet_2020,self).__init__() 
    self.bn1=torch.nn.BatchNorm2d(num_features=3) 
    self.conv1=torch.nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,stride=1,padding=0)
    self.act1=torch.nn.ReLU()
    self.drt1=torch.nn.Dropout2d(p=0.3)
    self.bn2=torch.nn.BatchNorm2d(num_features=16) 
    self.conv2=torch.nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3,stride=1,padding=0)
    self.act2=torch.nn.ReLU() 
    self.bn3=torch.nn.BatchNorm2d(num_features=16) 
    self.conv3=torch.nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3,stride=1,padding=0)
    self.act3=torch.nn.ReLU()
    self.pool1=torch.nn.MaxPool2d(kernel_size=2,stride=2)

    self.bn4=torch.nn.BatchNorm2d(num_features=16) 
    self.conv4=torch.nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=0)
    self.act4=torch.nn.ReLU()
    self.drt2=torch.nn.Dropout2d(p=0.3)
    self.bn5=torch.nn.BatchNorm2d(num_features=32) 
    self.conv5=torch.nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=0)
    self.act5=torch.nn.ReLU() 
    self.bn6=torch.nn.BatchNorm2d(num_features=32) 
    self.conv6=torch.nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=0)
    self.act6=torch.nn.ReLU()
    self.pool2=torch.nn.MaxPool2d(kernel_size=2,stride=2)

    self.bn7=torch.nn.BatchNorm2d(num_features=32) 
    self.conv7=torch.nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=0)
    self.act7=torch.nn.ReLU()
    self.drt3=torch.nn.Dropout2d(p=0.3)
    self.bn8=torch.nn.BatchNorm2d(num_features=64) 
    self.conv8=torch.nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1,padding=0)
    self.act8=torch.nn.ReLU() 
    self.bn9=torch.nn.BatchNorm2d(num_features=64) 
    self.conv9=torch.nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1,padding=0)
    self.act9=torch.nn.ReLU()
    self.pool3=torch.nn.MaxPool2d(kernel_size=2,stride=2)

    self.fc1=torch.nn.Linear(30976,702)
    self.act10=torch.nn.ReLU()
    self.drt4=torch.nn.Dropout(p=0.25)
    self.fc2=torch.nn.Linear(702,2)
    self.sm=torch.nn.Softmax(dim=1)
 def forward(self,x):
    x=self.bn1(x)
    x=self.conv1(x)
    x=self.act1(x)
    x=self.drt1(x)
    x=self.bn2(x) 
    x=self.conv2(x)
    x=self.act2(x) 
    x=self.bn3(x) 
    x=self.conv3(x)
    x=self.act3(x)
    x=self.pool1(x)

    x=self.bn4(x)
    x=self.conv4(x)
    x=self.act4(x)
    x=self.drt2(x)
    x=self.bn5(x) 
    x=self.conv5(x)
    x=self.act5(x) 
    x=self.bn6(x) 
    x=self.conv6(x)
    x=self.act6(x)
    x=self.pool2(x)

    x=self.bn7(x)
    x=self.conv7(x)
    x=self.act7(x)
    x=self.drt3(x)
    x=self.bn8(x) 
    x=self.conv8(x)
    x=self.act8(x) 
    x=self.bn9(x) 
    x=self.conv9(x)
    x=self.act9(x)
    x=self.pool3(x)

    x=x.view(x.size(0),x.size(1)*x.size(2)*x.size(3))
     
    x=self.fc1(x)
    x=self.act10(x)
    x=self.drt4(x)
    x=self.fc2(x)
    return x
 def vero(self,x):
       return self.sm(self.forward(x))     

In [0]:
Set=CatDogNet_2020()
Set.load_state_dict(torch.load('CatDogModelv6.pt',map_location=torch.device('cuda:0')))

In [0]:

def train_model(model, loss, optimizer, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    

Объявление модели + обработка данных

In [0]:
s=imread('/content/gdrive/Mydrive/датасет_тренировка/Кошка0.jpg')

print(s)
'''
Y=[]
s=imread('Кошка0'+".jpg")
s=s.reshape(1,s.shape[2],s.shape[0],s.shape[1])
s1=F.pad(torch.tensor(s,dtype=float),(500-s.shape[3],0,0,500-s.shape[2]))
t=imread('Собака0'+'.jpg')
t=t.reshape(1,t.shape[2],t.shape[0],t.shape[1])
t1=F.pad(torch.tensor(t,dtype=float),(500-t.shape[3],0,0,500-t.shape[2]))
Y.append(1)
Y.append(0)
L=torch.cat((s1,t1),dim=0)
for i in range(1,100):
    s=imread('Кошка'+str(i)+".jpg") 
    s=s.reshape(1,s.shape[2],s.shape[0],s.shape[1])
    s1=F.pad(torch.tensor(s,dtype=float),(500-s.shape[3],0,0,500-s.shape[2]))
    t=imread('Собака'+str(i)+'.jpg')
    t=t.reshape(1,t.shape[2],t.shape[0],t.shape[1])
    t1=F.pad(torch.tensor(t,dtype=float),(500-t.shape[3],0,0,500-t.shape[2]))
    Y.append(1)
    Y.append(0)
    L=torch.cat((L,torch.cat((s1,t1),dim=0)),dim=0)
Y=torch.LongTensor(Y)    
L/=255
print(L.size())
'''

валидационные недоданные

In [0]:

'''
Y1=[]
s=imread('Кошка100'+".jpg")
s=s.reshape(1,s.shape[2],s.shape[0],s.shape[1])
s1=F.pad(torch.tensor(s,dtype=float),(500-s.shape[3],0,0,500-s.shape[2]))
t=imread('Собака100'+'.jpg')
t=t.reshape(1,t.shape[2],t.shape[0],t.shape[1])
t1=F.pad(torch.tensor(t,dtype=float),(500-t.shape[3],0,0,500-t.shape[2]))
Y1.append(1)
Y1.append(0)
L1=torch.cat((s1,t1),dim=0)
for i in range(101,140):
    print(i)
    s=imread('Кошка'+str(i)+".jpg") 
    s=s.reshape(1,s.shape[2],s.shape[0],s.shape[1])
    s1=F.pad(torch.tensor(s,dtype=float),(500-s.shape[3],0,0,500-s.shape[2]))
    t=imread('Собака'+str(i)+'.jpg')
    t=t.reshape(1,t.shape[2],t.shape[0],t.shape[1])
    t1=F.pad(torch.tensor(t,dtype=float),(500-t.shape[3],0,0,500-t.shape[2]))
    Y1.append(1)
    Y1.append(0)
    L1=torch.cat((L1,torch.cat((s1,t1),dim=0)),dim=0)
Y1=torch.LongTensor(Y1)    
L1/=255
print(L1.size())
'''


Объявление функции потерь, оптимайзера, размера батча

In [0]:
loss=torch.nn.CrossEntropyLoss()
optim=torch.optim.Adam(Set.parameters(),lr=1.5e-5,weight_decay=0.01,amsgrad=True)
Set=Set.to(device)

In [0]:
from tqdm import tqdm

Цикл эпох обучения

In [0]:
train_model(Set,loss,optim,10)

In [0]:

'''
for i in range(60):
     chet=0
     order=np.random.permutation(len(L))
     order1=np.random.permutation(len(L1))
     Set.train()
     for start_index in range(0, len(L),batch_size):
         optim.zero_grad()
         batch_indexes=order[start_index:start_index+batch_size]
         x_batch=L[batch_indexes]
         y_batch=Y[batch_indexes]
         y_p=Set.forward(x_batch.float())
         loss_val=loss(y_p,y_batch)
         chet+=loss_val.data
         loss_val.backward()      
         optim.step() 
         optim.zero_grad()    
     print('Train_loss: ',chet/20)
     chet=0
     Set.eval()
     for start_index in range(0, len(L1),batch_size):
         batch_indexes=order1[start_index:start_index+batch_size]
         x_batch=L1[batch_indexes]
         y_batch=Y1[batch_indexes]
         y_p=Set.forward(x_batch.float())
         loss_val=loss(y_p,y_batch)
         chet+=loss_val.data
     print('Validation_loss: ',chet/8)
     '''


     
     
        

In [0]:
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_dir='/content/gdrive/My Drive/Датасет_тест3'
test_dataset = torchvision.datasets.ImageFolder(test_dir, test_transforms)
batch_size = 7
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
'''
def test_model(model, loss, optimizer):
     model.eval() 
     dataloader = test_dataloader
     running_loss = 0.
     running_acc = 0.

            # Iterate over data.
     for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                preds = model(inputs)
                loss_value = loss(preds, labels)
                preds_class = preds.argmax(dim=1)

                    

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

     epoch_loss = running_loss / len(dataloader)
     epoch_acc = running_acc / len(dataloader)
     print('{} Loss: {:.4f} Acc: {:.4f}'.format('test', epoch_loss, epoch_acc), flush=True)
test_model(Set,loss,optim)
'''

Проверка на тестовой выборке

In [0]:

torch.save(Set.state_dict(), 'CatDogModelv6.pt')

In [0]:
Set.eval()
X_batch, y_batch = next(iter(test_dataloader))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
plt.imshow(X_batch[0].permute(1, 2, 0).numpy() * std + mean)
k=Set.forward(X_batch[0].unsqueeze(0).to(device))
e=2.71828
if k[0][0]>k[0][1]:
  print('Кошка, c точностью:',(e**k[0][0]/(e**k[0][0]+e**k[0][1])).item())
else:
  print('Собака с точностью:',(e**k[0][1]/(e**k[0][0]+e**k[0][1])).item())  